<a href="https://colab.research.google.com/github/aekanun2020/2022-PSU-Data-Science-for-Tourism-Course1/blob/main/Quiz2-PySpark_DataScience_onColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
   .appName("Neural Network Model") \
   .config("spark.executor.memory", "3gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [ ]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
! head -3 testfunnel.csv

source,isTrueDirect,sourceKeyword,medium,isVideoAd,fullVisitorId,visitId,date,newVisits,hitReferer,hitType,hitAction_type,hitNumber,hitHour,hitMin,timeMicroSec,v2ProductName,productListName,isClick,isImpression,sessionQualityDim,timeOnScreen,timeOnSite,totalTransactionRevenue
google,true,(not provided),organic,,4988612949713423910,1489617360,20170315,,,PAGE,0,28,15,46,608771,Google Accent Insulated Stainless Steel Bottle,Category,,true,,,1897,
google,true,(not provided),organic,,4988612949713423910,1489617360,20170315,,,PAGE,0,27,15,46,605064,Google Pocket Bluetooth Speaker,Category,,true,,,1897,


In [ ]:
! wget https://storage.googleapis.com/sharewebinar/testfunnel.csv

--2021-08-26 07:39:04--  https://storage.googleapis.com/sharewebinar/testfunnel.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 142.251.2.128, 2607:f8b0:4023:c06::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1501012 (1.4M) [text/csv]
Saving to: ‘testfunnel.csv.6’

testfunnel.csv.6    100%[===================>]   1.43M  --.-KB/s    in 0.02s   

2021-08-26 07:39:04 (64.2 MB/s) - ‘testfunnel.csv.6’ saved [1501012/1501012]



In [ ]:
raw_df = spark.read.option('header','true')\
.option("inferSchema" , "true")\
.format('csv').load('./testfunnel.csv')

In [ ]:
raw_df.sample(0.001,seed=12).toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12
source,(direct),(direct),google,google,google,google,(direct),(direct),(direct),google,(direct),google,google
isTrueDirect,None,None,None,None,None,None,True,True,True,None,None,True,None
sourceKeyword,None,None,(not provided),(not provided),(not provided),(not provided),None,None,None,(not provided),None,(not provided),(not provided)
medium,(none),(none),organic,organic,organic,organic,(none),(none),(none),organic,(none),organic,organic
isVideoAd,None,None,None,None,None,None,None,None,None,None,None,None,None
fullVisitorId,4990075466656363861,4990257633001131707,4990697502077570091,4991105026836415438,4991962803971844700,4992499749003799871,4992819126088253676,4993613447705442396,4993613447705442396,4994855995081050813,4995356988620593823,4995584940873803432,4995903322841177843
visitId,1480486355,1485540848,1494612328,1488300417,1475876346,1472235854,1476938325,1482297870,1483430590,1495711333,1481560042,1497463936,1493223582
date,20161129,20170127,20170512,20170228,20161007,20160826,20161019,20161220,20170103,20170525,20161212,20170614,20170426
newVisits,1,1,1,1,1,1,1,NaN,NaN,1,1,NaN,1
hitReferer,None,https://www.google.nl/,None,None,None,None,None,None,None,https://www.google.ie/,None,None,None


1. ธุรกิจต้องการ ML Model ไปใช้ในการทำนายว่า ลูกค้าที่กำลังเลือกดูสินค้าอยู่ จะซื้อหรือไม่ซื้อสินค้า (รวมถึงพฤติกรรมอื่นๆ ด้วย)

In [ ]:
raw_df.count()

9999

2. Data Understanding

In [ ]:
len(raw_df.columns)

24

In [ ]:
raw_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
source,9999,None,None,(direct),youtube.com
sourceKeyword,4042,None,None,(not provided),youtube merch
medium,9999,None,None,(none),referral
fullVisitorId,9999,4.5067648825526333E18,1.39995873491150899E18,49898505875812980,4995924842281653133
visitId,9999,1.4850613864552455E9,9020666.664979974,1470106850,1501649190
date,9999,2.016601000310031E7,4694.944823979139,20160801,20170801
newVisits,7269,1.0,0.0,1,1
hitReferer,1333,None,None,http://images.google.co.in/imgres?imgurl=https...,https://www.youtube.com/yt/about/press/
hitType,9999,None,None,EVENT,PAGE


In [ ]:
raw_df.printSchema()

root
 |-- source: string (nullable = true)
 |-- isTrueDirect: boolean (nullable = true)
 |-- sourceKeyword: string (nullable = true)
 |-- medium: string (nullable = true)
 |-- isVideoAd: boolean (nullable = true)
 |-- fullVisitorId: long (nullable = true)
 |-- visitId: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- newVisits: integer (nullable = true)
 |-- hitReferer: string (nullable = true)
 |-- hitType: string (nullable = true)
 |-- hitAction_type: integer (nullable = true)
 |-- hitNumber: integer (nullable = true)
 |-- hitHour: integer (nullable = true)
 |-- hitMin: integer (nullable = true)
 |-- timeMicroSec: integer (nullable = true)
 |-- v2ProductName: string (nullable = true)
 |-- productListName: string (nullable = true)
 |-- isClick: boolean (nullable = true)
 |-- isImpression: boolean (nullable = true)
 |-- sessionQualityDim: integer (nullable = true)
 |-- timeOnScreen: string (nullable = true)
 |-- timeOnSite: integer (nullable = true)
 |-- totalTransac